## Download ERA5 data  
To compare the forecast data with the real observations, use reanalysis data. Therefore download these data from Copernicus Climate Store.

In [1]:
import cdsapi

In [2]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': 'total_precipitation',
        'year': '2024',
        'month': '05',
        'day': [
            '18', '19',
        ],
        'time': [
            '00:00', '06:00', '12:00',
            '18:00',
        ],
    },
    'era5_18_19may.nc')

2024-07-11 07:18:57,803 INFO Welcome to the CDS
2024-07-11 07:18:57,804 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-11 07:18:57,934 INFO Request is queued
2024-07-11 07:18:58,971 INFO Request is running
2024-07-11 07:19:06,204 INFO Request is completed
2024-07-11 07:19:06,206 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.internal-1720682342.3137634-26924-16-260bebec-7c2a-4e33-b039-8e43d3522ab1.nc to era5_18_19may.nc (15.9M)
2024-07-11 07:19:07,502 INFO Download rate 12.2M/s  


Result(content_length=16621584,content_type=application/x-netcdf,location=https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.internal-1720682342.3137634-26924-16-260bebec-7c2a-4e33-b039-8e43d3522ab1.nc)

In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
dt=xr.open_dataset('era5_18_19may.nc')

In [4]:
dt

<xarray.Dataset> Size: 66MB
Dimensions:    (longitude: 1440, latitude: 721, time: 8)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 64B 2024-05-18 ... 2024-05-19T18:00:00
Data variables:
    tp         (time, latitude, longitude) float64 66MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 07:19:02 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

Compare ERA5 with Forecast:  
regrid ERA5 on same grid  
calculate difference between them  
mean & std etc